In [1]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv("PreprocessedDataTrain.csv")
test = pd.read_csv("PreprocessedDataTest.csv")

In [4]:
train.head()

,Id,description,gender,Category,jobtitle,cleaned_Data
0,0,She is also a Ronald D. Asmus Policy Entrepre...,F,19,professor,also ronald asmus policy entrepreneur fellow g...
1,1,He is a member of the AICPA and WICPA. Brent ...,M,9,accountant,member aicpa wicpa brent graduated university ...
2,2,Dr. Aster has held teaching and research posi...,M,19,professor,dr aster held teaching research positions ben ...
3,3,He runs a boutique design studio attending cl...,M,24,architect,runs boutique design studio attending clients ...
4,4,"He focuses on cloud security, identity and ac...",M,24,architect,focuses cloud security identity access managem...


In [5]:
test.head()

,Id,description,gender,cleaned_Data
0,0,She currently works on CNN’s newest primetime...,F,currently works cnns newest primetime show par...
1,1,Lavalette’s photographs have been shown widel...,M,lavalettes photographs shown widely editorial ...
2,2,Along with his academic and professional deve...,M,along academic professional development gabrie...
3,3,She obtained her Ph.D. in Islamic Studies at ...,F,obtained phd islamic studies duke university s...
4,4,She studies issues of women and Islam and has...,F,studies issues women islam written polygamy am...


In [12]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import transformers
import tqdm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold


In [13]:
#%matplotlib inline

# fix random seed for reproducibility
seed = 23
np.random.seed(seed)
tf.random.set_seed(seed)

In [14]:
df_mislabeled = train.groupby(['cleaned_Data']).nunique().sort_values(by='jobtitle', ascending=False)
df_mislabeled = df_mislabeled[df_mislabeled['jobtitle'] > 1]['jobtitle']
df_mislabeled = df_mislabeled.index.tolist()

In [15]:
drop_dup_rows = train['cleaned_Data'].isin(df_mislabeled)
train = train[~drop_dup_rows]

In [16]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [17]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=23)

In [18]:
X,y=train['cleaned_Data'].values,train['Category'].values

for tr_idx,te_idx in sss.split(X,y):
  X_train = X[tr_idx]
  y_train = y[tr_idx]
  X_test = X[te_idx]
  y_test = y[te_idx]

In [19]:
def create_bert_input_features(tokenizer, docs, max_seq_length):
  
  all_ids, all_masks, all_segments= [], [], []
  for doc in tqdm.tqdm(docs, desc="Converting docs to features"):
      
      tokens = tokenizer.tokenize(doc)
      
      if len(tokens) > max_seq_length-2:
          tokens = tokens[0 : (max_seq_length-2)]
      tokens = ['[CLS]'] + tokens + ['[SEP]']
      ids = tokenizer.convert_tokens_to_ids(tokens)
      masks = [1] * len(ids)
      
      # Zero-pad up to the sequence length.
      while len(ids) < max_seq_length:
          ids.append(0)
          masks.append(0)
          
      segments = [0] * max_seq_length
      all_ids.append(ids)
      all_masks.append(masks)
      all_segments.append(segments)
      
  encoded = np.array([all_ids, all_masks, all_segments])
  
  return encoded

In [20]:

MAX_SEQ_LENGTH = 60

inp_id = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_ids")
inp_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_masks")
inp_segment = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_segment_ids")
inputs = [inp_id, inp_mask, inp_segment]

In [21]:
hidden_state = transformers.TFBertModel.from_pretrained('bert-base-uncased')(inputs)
pooled_output = hidden_state[1]
dense1 = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(256, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)
output = tf.keras.layers.Dense(28, activation='softmax')(drop2)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [22]:
model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.load_weights('C:/Users/yoges/MachineLearningProject/bert_t.h5')


OSError: Unable to open file (unable to open file: name = 'C:/Users/yoges/MachineLearningProject/bert_t.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [24]:
print(model.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bert_input_ids (InputLayer)     [(None, 60)]         0                                            
__________________________________________________________________________________________________
bert_input_masks (InputLayer)   [(None, 60)]         0                                            
__________________________________________________________________________________________________
bert_segment_ids (InputLayer)   [(None, 60)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 60, 768), (N 109482240   bert_input_ids[0][0]             
                                                                 bert_input_masks[0][0]

In [25]:
X_val = X_train[:5000]
y_val = y_train[:5000]
X_train_ = X_train[5000:]
y_train_ = y_train[5000:]

In [26]:
train_features_ids, train_features_masks, train_features_segments = create_bert_input_features(tokenizer, 
                                                                                               X_train_, 
                                                                                               max_seq_length=MAX_SEQ_LENGTH)
val_features_ids, val_features_masks, val_features_segments = create_bert_input_features(tokenizer, 
                                                                                         X_val, 
                                                                                         max_seq_length=MAX_SEQ_LENGTH)
#test_features = create_bert_input_features(tokenizer, test_reviews, max_seq_length=MAX_SEQ_LENGTH)
print('Train Features:', train_features_ids.shape, train_features_masks.shape, train_features_segments.shape)
print('Val Features:', val_features_ids.shape, val_features_masks.shape, val_features_segments.shape)

check_point = tf.keras.callbacks.ModelCheckpoint('/home/deepakumar_msc/check_point/bert_t_1.h5',save_best_only=True,save_weights_only=True)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

model.fit([train_features_ids, 
           train_features_masks, 
           train_features_segments], y_train_, 
          validation_data=([val_features_ids, 
                            val_features_masks, 
                            val_features_segments], y_val),
          epochs=1, 
          batch_size=13, 
          callbacks=[check_point,early_stopping],
          shuffle=True,verbose=1)#,
          #verbose=1)

print('learning is over')


test_features_ids, test_features_masks, test_features_segments = create_bert_input_features(tokenizer, 
                                                                                         X_test, 
                                                                                         max_seq_length=MAX_SEQ_LENGTH)

y_hat = model.predict([test_features_ids,test_features_masks,test_features_segments], verbose=1)

y_hat = np.argmax(y_hat,axis=1)

print('initial pred')

print(classification_report(y_test,y_hat))

Converting docs to features: 100%|████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:04<00:00, 1114.41it/s]


Train Features: (157877, 60) (157877, 60) (157877, 60)
Val Features: (5000, 60) (5000, 60) (5000, 60)
12145/12145 [==============================] - 2434s 200ms/step - loss: 0.7995 - accuracy: 0.7829 - val_loss: 0.5818 - val_accuracy: 0.8372


Converting docs to features:   0%|▎                                                                           | 213/54293 [00:00<00:50, 1081.18it/s]

learning is over


Converting docs to features: 100%|██████████████████████████████████████████████████████████████████████████| 54293/54293 [00:49<00:00, 1087.18it/s]


1697/1697 [==============================] - 229s 135ms/step
initial pred
              precision    recall  f1-score   support

           0       0.67      0.67      0.67       374
           1       0.90      0.71      0.79      1029
           2       0.62      0.86      0.72       236
           3       0.62      0.60      0.61      2286
           4       0.79      0.70      0.74       202
           5       0.84      0.80      0.82      1155
           6       0.76      0.76      0.76      3074
           7       0.63      0.76      0.69       214
           8       0.86      0.68      0.76      1653
           9       0.82      0.76      0.79       780
          10       0.72      0.73      0.73       208
          11       0.71      0.77      0.73      2901
          12       0.83      0.80      0.82       410
          13       0.72      0.75      0.73      1015
          14       0.95      0.79      0.86      3155
          15       0.81      0.74      0.77      1073
       